<a href="https://colab.research.google.com/github/sensationalspace/colab/blob/main/RAG_Improvments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninsta

In [ ]:
!pip install pypdf
!pip install openai

In [ ]:
import openai

openai.api_key = "YOUR_KEY_HERE"

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.tools import QueryEngineTool, ToolMetadata


documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine();

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="basic_rules",
            description=(
                "the basic game rules and description"
            ),
        ),
    ),
]

In [ ]:
from llama_index.agent import ContextRetrieverOpenAIAgent

context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    query_engine_tools,
    index.as_retriever(similarity_top_k=3),
    verbose=True,
)

In [ ]:
response = context_agent.chat("What is the game about?")

print(response)

Context information is below.
---------------------
4
GoalGoal
Y our goal is to spend all your money as fast as you can, in 
accordance with your uncle's last will.
The Round The Round
A game will have at most 7 rounds. Each round has 5 phases:
1. Setup2. Planning3. Errands4. Actions5. End of RoundIf any player manages to get rid of all their money during 
any round of the game, the round is still played in full.1. Setup 1. Setup
First you need to know what opportunities are available.
Deal one card onto each space of the card off  ering board. 
Th e icons indicate which type of card should be dealt where. 
On some spaces, the same deck will be used every round. On others, the deck changes as the game progresses.
1st and 2nd round,
property card
3rd and subsequent 
rounds, event card
In the fi  rst round, after the setup, each player chooses only 
two cards from their hand to keep and discards the rest, face--down, to the appropriate discard pile.
Example of the setup of the ﬁ rst roun

In [ ]:
response = context_agent.chat("Give me in a short answer how to play the first turn?")

print(response)

Context information is below.
---------------------
5
2. Planning 2. Planning
Th is is when you decide how many opportunities you want to consider , 
how many errands you need to have done for you, and how much of the day you will set aside for enjoying your money.
Beginning with the starting player and continuing clockwise, 
each player makes a plan for the day by placing his or her planning marker on one of the hourglass spaces of the planning board.
Y our choice determines:
 d Th e number of cards you draw.
(Y ou take them immediately.)
 d Th e number of errand boys you will
have in the errand phase.
 d Th e number of actions you will have
in the action phase.
 d Th e order in which you will play
for the remainder of the round.
As soon as you declare your plan, draw the indicated number 
of cards. Y ou may draw cards in any combination from the tops of the 4 regular decks. Y ou cannot draw from the deck of special cards (with crowns on the backs). Y ou cannot draw from the cards on 

# Cost Optimization

In [ ]:
from llama_index.postprocessor import SentenceEmbeddingOptimizer


query_engine = index.as_query_engine(
    node_postprocessors=[SentenceEmbeddingOptimizer(percentile_cutoff=0.5)]
)

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="basic_rules",
            description=(
                "the basic game rules and description"
            ),
        ),
    ),
]

context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    query_engine_tools,
    index.as_retriever(similarity_top_k=3),
    verbose=True,
)

response = context_agent.chat("Give me in a short answer how to play the first turn?")

print(response)

Context information is below.
---------------------
5
2. Planning 2. Planning
Th is is when you decide how many opportunities you want to consider , 
how many errands you need to have done for you, and how much of the day you will set aside for enjoying your money.
Beginning with the starting player and continuing clockwise, 
each player makes a plan for the day by placing his or her planning marker on one of the hourglass spaces of the planning board.
Y our choice determines:
 d Th e number of cards you draw.
(Y ou take them immediately.)
 d Th e number of errand boys you will
have in the errand phase.
 d Th e number of actions you will have
in the action phase.
 d Th e order in which you will play
for the remainder of the round.
As soon as you declare your plan, draw the indicated number 
of cards. Y ou may draw cards in any combination from the tops of the 4 regular decks. Y ou cannot draw from the deck of special cards (with crowns on the backs). Y ou cannot draw from the cards on 

In [ ]:
response = context_agent.chat("How can i win the game?")

print(response)

Context information is below.
---------------------
4
GoalGoal
Y our goal is to spend all your money as fast as you can, in 
accordance with your uncle's last will.
The Round The Round
A game will have at most 7 rounds. Each round has 5 phases:
1. Setup2. Planning3. Errands4. Actions5. End of RoundIf any player manages to get rid of all their money during 
any round of the game, the round is still played in full.1. Setup 1. Setup
First you need to know what opportunities are available.
Deal one card onto each space of the card off  ering board. 
Th e icons indicate which type of card should be dealt where. 
On some spaces, the same deck will be used every round. On others, the deck changes as the game progresses.
1st and 2nd round,
property card
3rd and subsequent 
rounds, event card
In the fi  rst round, after the setup, each player chooses only 
two cards from their hand to keep and discards the rest, face--down, to the appropriate discard pile.
Example of the setup of the ﬁ rst roun

#Reranking

In [ ]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
)

from llama_index.llms import OpenAI

documents = SimpleDirectoryReader("data").load_data()

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)


In [ ]:
from llama_index.postprocessor import LLMRerank

reranker = LLMRerank(
        choice_batch_size=5,
        top_n=3,
        service_context=service_context,
  )

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[reranker],
    response_mode="refine",
)

In [ ]:
response = query_engine.query(
    "What is the best strategy for the first turns?",
)

print(response)

When starting the game, it is important to consider your priorities and goals. Determine how many errands you want to pursue, how many tasks you need assistance with, and how much time you want to allocate for leisure activities. These factors will influence the number of errand boys you should place, the number of errands you will be able to perform, and the sequence in which you will play for the rest of the round. It is crucial to make these decisions without knowing the specific errand opportunities available.


In [ ]:
response = query_engine.query(
    "What are the basic rules?",
)

print(response)

The basic rules of the game involve using cards to take actions and spending all your money as quickly as possible according to your uncle's last will. The game consists of up to 7 rounds, each with 5 phases: Setup, Planning, Errands, Actions, and End of Round. During Setup, players are dealt cards onto the card offering board, with different types of cards being used in different rounds. In the first round, players choose two cards to keep from their hand and discard the rest.
